In [35]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout, LSTM 
from keras.optimizers import RMSprop 
import numpy as np
import random
import sys

In [36]:
import string

def clean_text(raw_text):
    translator = str.maketrans('', '', string.punctuation)
    return raw_text.translate(translator)

filename = "Trading with Mexico.txt"
raw_text = open(filename, 'r', encoding='utf-8').read().lower()
raw_text = clean_text(raw_text)


In [37]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [38]:
maxlen = 60 
step = 3
chars = sorted(list(set(raw_text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [39]:
sentences = []
next_chars = []

for i in range(0, len(raw_text) - maxlen, step):
    sentences.append(raw_text[i: i + maxlen])
    next_chars.append(raw_text[i + maxlen])

print("Number of sequences:", len(sentences))

Number of sequences: 127146


In [40]:
import numpy as np

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
import sys
import random
import numpy as np

# Dictionary to store outputs: {epoch: {temperature: generated_text}}
generated_texts = {}

temperatures = [0.2, 0.5, 1.0, 1.2]
epochs = 60

for epoch in range(1, epochs + 1):
    print(f"\n{'='*40}\nEpoch {epoch}\n{'='*40}")
    
    model.fit(X, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(raw_text) - maxlen - 1)
    sentence = raw_text[start_index: start_index + maxlen]
    generated = sentence

    generated_texts[epoch] = {}  # Store texts per epoch

    for temp in temperatures:
        sys.stdout.write(f"\n--- Temperature: {temp}\n")
        sys.stdout.write(f'Seed: "{sentence}"\n')
        
        gen_text = sentence  # Start with the seed

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature=temp)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char
            gen_text += next_char

        generated_texts[epoch][temp] = gen_text

with open("generated_outputs.txt", "w", encoding="utf-8") as f:
    for epoch in sorted(generated_texts.keys()):
        f.write(f"\n{'='*50}\nEpoch {epoch}\n{'='*50}\n")
        for temp, text in generated_texts[epoch].items():
            f.write(f"\n--- Temperature: {temp}\n{text}\n")



Epoch 1
  1/994 ━━━━━━━━━━━━━━━━━━━━ 1:39 100ms/step - loss: 2.0252

2025-06-12 01:10:22.212133: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 442468080 exceeds 10% of free system memory.


994/994 ━━━━━━━━━━━━━━━━━━━━ 91s 92ms/step - loss: 2.1244

--- Temperature: 0.2
Seed: "es

the answer to this proposition has been astonishingly to"

--- Temperature: 0.5
Seed: "d the prices and the dand the denting the fore the mexicon a"

--- Temperature: 1.0
Seed: "in the promere of the more of the erevers and the mexican co"

--- Temperature: 1.2
Seed: "ct of lirgition thacs ass ow the are
dacas ta “romuituals tr"

Epoch 2


2025-06-12 01:13:14.798275: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 442468080 exceeds 10% of free system memory.


994/994 ━━━━━━━━━━━━━━━━━━━━ 91s 91ms/step - loss: 2.0033

--- Temperature: 0.2
Seed: "he absence of navigable streams
the mere possibility of mexi"

--- Temperature: 0.5
Seed: "xican the mexican the mexico in the race the propertion the "

--- Temperature: 1.0
Seed: "he more in the the state the
prout of the propures of the th"

--- Temperature: 1.2
Seed: "f of the tistutany wor heone not in tiy lissreas it tring ta"

Epoch 3
  1/994 ━━━━━━━━━━━━━━━━━━━━ 1:53 114ms/step - loss: 1.9796

2025-06-12 01:16:06.667833: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 442468080 exceeds 10% of free system memory.


994/994 ━━━━━━━━━━━━━━━━━━━━ 91s 92ms/step - loss: 1.9263

--- Temperature: 0.2
Seed: "he edict of september 26
1915 abolishing out of hand the hue"

--- Temperature: 0.5
Seed: " the mexican south and the mexican and the propert of the pr"

--- Temperature: 1.0
Seed: "are the gralus of the ere the peract of the seated of mexico"

--- Temperature: 1.2
Seed: "emeats as thimain 1“30 a deve nece aqleine werc landous the "

Epoch 4
994/994 ━━━━━━━━━━━━━━━━━━━━ 90s 90ms/step - loss: 1.8622

--- Temperature: 0.2
Seed: "ging under the
revolutionary presidents of recent years but "

--- Temperature: 0.5
Seed: "e propertion of the mexico in the propertion of the reans an"

--- Temperature: 1.0
Seed: "g the great of state devernces and which of butking and the "

--- Temperature: 1.2
Seed: "ofercabeon live the swill facted and war forgation thend evl"

Epoch 5
994/994 ━━━━━━━━━━━━━━━━━━━━ 88s 88ms/step - loss: 1.8051

--- Temperature: 0.2
Seed: "any inc


                       printed in the u s